### **🔥🔥DiffBindFR**
<div class="alert alert-info"> Diffusion model based flexible protein-ligand docking
</div>

for more details see our [Preprint](https://arxiv.org/abs/2311.15201)

Here, we conduct a demo by using DiffBindFR to redock ligand (fetched by PDB ID: 2ZEC) into the predefined pocket of **AlphaFold2 modelled structure** (Uniprot ID: Q15661).

In [1]:
import os, sys, glob
from pathlib import Path
home = os.path.join(os.getcwd(), '../')
home = os.path.abspath(home)
if home not in sys.path:
    sys.path.insert(0, home)
home = Path(home)
import pandas as pd
from rdkit import Chem
import MDAnalysis as mda
import nglview as nv
from nglview.color import ColormakerRegistry
from DiffBindFR import common
from DiffBindFR.evaluation import get_traj_id, export_xtc
from DiffBindFR.app.predict import runner
from DiffBindFR.utils import (
    pair_spatial_metrics, 
    PDBPocketResidues, 
    to_complex_block,
    read_molblock,
    update_mol_pose,
)

In [2]:
example_path = home / 'examples' / 'AF2'
holo = example_path / '2zec.pdb'
crystal_ligand = example_path / 'ligand.sdf'
af2 = example_path / 'Q15661_AF2.pdb'

#### PoseView of Holo structure

Here, pocket residues within 5 angstrom of crystal ligand are visualized (colored by <font color='red'>red</font>)

In [3]:
pocket_buffer = 5
holo_pocket = PDBPocketResidues.RDmolPocketResidues(
    str(holo), str(crystal_ligand),
)
view = holo_pocket.visualize_pocket(pocket_buffer)
view._remote_call('setSize', target='Widget', args=['','600px'])
view

NGLWidget()

#### Pocket Conformation visual inspection

Compare the AF2 modeled pocket conformation (<font color='yellow'>yellow cartoon</font> and <font color='blue'>blue sticks</font>) with the crystal structure in advance.

We could get the knowledge:

- AF2 modeled structure has holo-like backbone with CA RMSD = 0.32 A
- There are significant differences in pocket side chain conformation with sc-RMSD = 1.24 A, mainly from A:218:ASP, A:219:SER, A:221:GLN, A:244:TRP, A:246:GLU

In [4]:
view = holo_pocket.compare(str(af2))
view._remote_call('setSize', target='Widget', args=['','600px'])
view

NGLWidget()

In [4]:
# Quantitative comparison of pocket conformation between af2 structure and holo
holo_chainid = 'A' # see the receptor chain ID in poseview
results_df = pair_spatial_metrics(
    str(holo), str(crystal_ligand), str(af2),
    holo_chainid, 'A', # af2 chain id is A as it is monomer prediction
    bs_cutoff = pocket_buffer,
)
ca_rmsd = results_df.iloc[0].mean_ca_rmsd
sc_rmsd = results_df.iloc[0].mean_sc_rmsd
print('pocket CA RMSD within 5A of ligand:', round(ca_rmsd, 2))
print('pocket side chain heavy atoms RMSD within 5A of ligand:', round(sc_rmsd, 2))

pocket CA RMSD within 5A of ligand: 0.32
pocket side chain heavy atoms RMSD within 5A of ligand: 1.24


In [5]:
# let's see the key residues
flexible_residues = 'A:218:ASP,A:219:SER,A:221:GLN,A:244:TRP,A:246:GLU'
holo_chainid = 'A' # see the receptor chain ID in poseview
# reverse the input as the residue is on af2 structure and crystal_ligand here is useless
results_df = pair_spatial_metrics(
    str(af2), str(crystal_ligand), str(holo), 
    'A', holo_chainid, 
    bs_res_str = flexible_residues.split(','),
)
ca_rmsd = results_df.iloc[0].mean_ca_rmsd
sc_rmsd = results_df.iloc[0].mean_sc_rmsd
print(f'pocket CA RMSD of {flexible_residues}:', round(ca_rmsd, 2))
print(f'pocket side chain heavy atoms RMSD of {flexible_residues}:', round(sc_rmsd, 2))

pocket CA RMSD of A:218:ASP,A:219:SER,A:221:GLN,A:244:TRP,A:246:GLU: 0.23
pocket side chain heavy atoms RMSD of A:218:ASP,A:219:SER,A:221:GLN,A:244:TRP,A:246:GLU: 1.78


#### Run demo

So here, we would like to use **DiffBindFR** to perform flexible docking and dock the ligand into pocket and refine the side chain conformation so that the refined structure is close to holo.

In [6]:
experiment_name = 'Q15661'
export_dir = 'demo_af2_docking'
export_dir = os.path.abspath(export_dir)
seed = 888 # 8888 has better ligand rmsd but not sc RMSD

In [7]:
# input parameters in jupyter using argparse
parser = common.parse_args()
args = parser.parse_args(
    [
        '-l', str(crystal_ligand),
        '-p', str(af2),
        '-o', export_dir,
        '-np', '40', 
        '-gpu', '0', 
        '-cpu', '1', 
        '-bs', '16', 
        '-eval', '-rp', # here we automatically evaluate the redock performance
        '-cl', 
        '-st',
        '-n', experiment_name,
        '--seed', str(seed),
    ]
)
args.cfg_options = None
job_df = common.make_inference_jobs(args)
runner(job_df, args)

 ____  _  __  __ ____  _           _ _____ ____  
|  _ \(_)/ _|/ _| __ )(_)_ __   __| |  ___|  _ \ 
| | | | | |_| |_|  _ \| | '_ \ / _` | |_  | |_) |
| |_| | |  _|  _| |_) | | | | | (_| |  _| |  _ < 
|____/|_|_| |_| |____/|_|_| |_|\__,_|_|   |_| \_\ 


2024-03-13 22:17:38,637 - DiffBindFR - INFO - `use_rank_shift` = False,Set random seed to 888, deterministic: False
2024-03-13 22:17:38,672 - DiffBindFR - INFO - Start to prepare job (experiment name: Q15661).
2024-03-13 22:17:42,885 - DiffBindFR - INFO - dock Status: Prep task is Done!


Use Background Generator supported dataloader.
Initializing diffusion model...


2024-03-13 22:17:46,597 - DiffBindFR - INFO - load checkpoint from local path: /data01/zhujintao/projects/home/research/DiffBindFR_OpenSource/DiffBindFR/weights/diffbindfr_paper.pth
2024-03-13 22:17:48,655 - DiffBindFR - INFO - Running model inference...


[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 40/40, 0.5 task/s, elapsed: 76s, ETA:     0s

...

+-------------------------------------------------------------------------------------+
|                             DiffBindFR Model evaluations                            |
+-----------------------------------------------------------------------+-------------+
| Metric                                                                | Performance |
+-----------------------------------------------------------------------+-------------+
| Total poses                                                           | 1x40=40     |
| l-rmsd gold standard                                                  | 2.0         |
| centroid gold standard                                                | 1.0         |
| chi1_15 gold standard                                                 | 0.75        |
| sc-rmsd gold standard                                                 | 1.0         |
|                                                                       |             |
| All poses l-rmsd mean-std     

2024-03-13 22:19:36,926 - DiffBindFR - INFO - Start to correct error...


INFO: Pandarallel will run on 1 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


2024-03-13 22:19:52,701 - DiffBindFR - INFO - Error correction is completed.
2024-03-13 22:19:53,948 - DiffBindFR - INFO - Start to MDN based pose scoring...
Processing 40 jobs, chunk 50000 per iter, total 1 iters...: 100%|██████████| 1/1 [00:05<00:00,  5.39s/it]
2024-03-13 22:20:00,014 - DiffBindFR - INFO - Load scoring model...
100%|██████████| 3/3 [00:00<00:00, 14.51it/s]
2024-03-13 22:20:00,429 - DiffBindFR - INFO - Model Scoring is Done!
2024-03-13 22:20:00,469 - DiffBindFR - INFO - MDN based pose scoring is completed.
2024-03-13 22:20:00,471 - DiffBindFR - INFO - Clean up datasets and model_out.pt.
2024-03-13 22:20:00,475 - DiffBindFR - INFO - DiffBindFR docking is Done!


Total number of DiffBindFR perfect poses: 1
The DiffBindFR perfect selected L-RMSD success rate: 100.0 %
The DiffBindFR perfect selected sc-RMSD success rate: 100.0 %
The DiffBindFR perfect selected centroid success rate: 100.0 %
The DiffBindFR perfect selected chi1_15 success rate: 100.0 %

Use error corrected poses...
Total number of DiffBindFR-Smina perfect poses: 1
The DiffBindFR-Smina perfect selected success rate: 100.0 %
The DiffBindFR-Smina perfect selected sc-rmsd success rate: 100.0 %
The DiffBindFR-Smina perfect selected centroid success rate: 100.0 %
The DiffBindFR-Smina perfect selected chi1_15 success rate: 100.0 %

Total number of DiffBindFR-Smina top1 poses: 1
The DiffBindFR-Smina top1 selected success rate: 100.0 %
The DiffBindFR-Smina top1 selected sc-rmsd success rate: 100.0 %
The DiffBindFR-Smina top1 selected centroid success rate: 100.0 %
The DiffBindFR-Smina top1 selected chi1_15 success rate: 100.0 %

Total number of DiffBindFR-MDN top1 poses: 1
The DiffBindFR-M

In [8]:
results_dir = os.path.join(export_dir, experiment_name, 'results')
smina_top1 = os.path.join(results_dir, 'results_ec_smina_top1.csv')
smina_top1 = pd.read_csv(smina_top1)
smina_top1 = smina_top1.iloc[0]
smina_top1_protein = smina_top1.protein_pdb
smina_top1_pose = smina_top1.docked_lig
mdn_top1 = os.path.join(results_dir, 'results_ec_mdn_top1.csv')
mdn_top1 = pd.read_csv(mdn_top1)
mdn_top1 = mdn_top1.iloc[0]
mdn_top1_protein = mdn_top1.protein_pdb
mdn_top1_pose = mdn_top1.docked_lig

#### Smina top1 prediction

In [9]:
holo_pocket = PDBPocketResidues.RDmolPocketResidues(
    str(holo), str(crystal_ligand),
)
view = holo_pocket.visualize_pocket(pocket_buffer)
view = holo_pocket.compare(smina_top1_protein, ligand_sdf = smina_top1_pose)
view._remote_call('setSize', target='Widget', args=['','600px'])
view

NGLWidget()

In [12]:
print('DiffBindFR-Smina')
print(f'ligand RMSD:', round(smina_top1['l-rmsd_ec'], 2))

results_df = pair_spatial_metrics(
    str(holo), str(crystal_ligand), str(smina_top1_protein),
    holo_chainid, 'A', # af2 chain id is A as it is monomer prediction
    bs_cutoff = pocket_buffer,
)
sc_rmsd = results_df.iloc[0].mean_sc_rmsd
print('pocket side chain heavy atoms RMSD within 5A of ligand:', round(sc_rmsd, 2))

results_df = pair_spatial_metrics(
    str(smina_top1_protein), str(crystal_ligand), str(holo), 
    'A', holo_chainid, 
    bs_res_str = flexible_residues.split(','),
)
sc_rmsd = results_df.iloc[0].mean_sc_rmsd
print(f'pocket side chain heavy atoms RMSD of {flexible_residues}:', round(sc_rmsd, 2))

DiffBindFR-Smina
ligand RMSD: 0.87
pocket side chain heavy atoms RMSD within 5A of ligand: 1.18
pocket side chain heavy atoms RMSD of A:218:ASP,A:219:SER,A:221:GLN,A:244:TRP,A:246:GLU: 0.97


In [13]:
print('DiffBindFR-MDN')
print(f'ligand RMSD:', round(mdn_top1['l-rmsd_ec'], 2))

results_df = pair_spatial_metrics(
    str(holo), str(crystal_ligand), str(mdn_top1_protein),
    holo_chainid, 'A', # af2 chain id is A as it is monomer prediction
    bs_cutoff = pocket_buffer,
)
sc_rmsd = results_df.iloc[0].mean_sc_rmsd
print('pocket side chain heavy atoms RMSD within 5A of ligand:', round(sc_rmsd, 2))

results_df = pair_spatial_metrics(
    str(mdn_top1_protein), str(crystal_ligand), str(holo), 
    'A', holo_chainid, 
    bs_res_str = flexible_residues.split(','),
)
sc_rmsd = results_df.iloc[0].mean_sc_rmsd
print(f'pocket side chain heavy atoms RMSD of {flexible_residues}:', round(sc_rmsd, 2))

DiffBindFR-MDN
ligand RMSD: 1.82
pocket side chain heavy atoms RMSD within 5A of ligand: 1.47
pocket side chain heavy atoms RMSD of A:218:ASP,A:219:SER,A:221:GLN,A:244:TRP,A:246:GLU: 1.42


#### Make trajectory movie

In [14]:
cm = ColormakerRegistry
cm.add_scheme_func('lig_atomwise','''
 this.atomColor = function (atom) {
     if (atom.element == "C") {
       return 0x7272e6 // C
     } else if (atom.element == "H") {
       return 0xecf0f1
     } else if (atom.element == "S") {
       return 0xf1c40f
     } else if (atom.element == "N") {
       return 0x2d2de1
     } else if (atom.element == "O") {
       return 0xff5252
     }
 }
''')
cm.add_scheme_func('prot_atomwise','''
 this.atomColor = function (atom) {
     if (atom.element == "C") {
       return 0xf9f902 // C
     } else if (atom.element == "H") {
       return 0xecf0f1
     } else if (atom.element == "S") {
       return 0xf1c40f
     } else if (atom.element == "N") {
       return 0x2d2de1
     } else if (atom.element == "O") {
       return 0xff5252
     }
 }
''')

def add_ec_to_xtc(
    sample_dir: str,
    topology: str,
    new_name: str = 'new_prl_traj.xtc',
) -> str:
    pdb_final = os.path.join(sample_dir, 'prot_final.pdb')
    lig_final = os.path.join(sample_dir, 'lig_final_ec.sdf')
    lig_final_mol = Chem.SDMolSupplier(lig_final)[0]
    lig_final_mol = Chem.MolFromPDBBlock(Chem.MolToPDBBlock(lig_final_mol))

    traj_dir = os.path.join(sample_dir, 'prl_traj')
    trajs = list(Path(traj_dir).glob('prl_*.pdb'))
    assert len(trajs) > 0, 'please export trajectory when you run DiffBindFR sampling by turn on -st.'
    ids = []
    for traj in trajs:
        stem = traj.stem
        traj_id = get_traj_id(stem)
        ids.append(traj_id)
    max_id = max(ids)
    final_id = max_id + 1
    final_traj_path = os.path.join(traj_dir, f'prl_{final_id}.pdb')
    seed_traj_path = trajs[0]

    mol_seed_block = read_molblock(seed_traj_path)
    mol_seed = Chem.MolFromPDBBlock(mol_seed_block) # use mol_seed topology to export PDB block
    lig_final_mol = update_mol_pose(mol_seed, lig_final_mol)

    trajectory = os.path.join(sample_dir, new_name)
    p_pdbblock = Path(pdb_final).read_text()
    l_pdbblock = Chem.MolToPDBBlock(lig_final_mol)
    try:
        complex_pdb_block = to_complex_block(p_pdbblock, l_pdbblock, final_traj_path)
        export_xtc(
            topology,
            traj_dir,
            trajectory,
        )
    finally:
        if os.path.exists(final_traj_path):
            os.remove(final_traj_path) # avoid increment by multiple run
    return trajectory

def show_nv_traj(
    sample_dir: str,
    repr_sel: str,
    add_ec_to_xtc_flag = True,
):
    topology = os.path.join(sample_dir, '../prl_topol.pdb')

    if add_ec_to_xtc_flag:
        # add ec ligand into xtc
        trajectory = add_ec_to_xtc(sample_dir, topology)
    else:
        trajectory = os.path.join(sample_dir, 'prl_traj.xtc')

    u = mda.Universe(topology, trajectory)
    system = u.select_atoms('all')
    t = nv.MDAnalysisTrajectory(system)
    w = nv.NGLWidget(t)
    w.clear_representations()
    w.add_cartoon(colorScheme = 'sstruc')
    w.add_representation(
        repr_type='ball+stick', 
        selection='[UNL]', # ligand resname
        color_scheme = 'lig_atomwise'
    )
    w.add_representation('licorice', selection=repr_sel, color_scheme='prot_atomwise')

    if add_ec_to_xtc_flag:
        os.remove(trajectory)
    
    return w

# make nglview selection expression
flex_residue_list = flexible_residues.split(',')
flex_resnumber = [x.split(':')[1] for x in flex_residue_list]
flex_resnumber = ':A and ' + '( ' + ' or '.join(flex_resnumber) + ' )'
flex_resnumber

':A and ( 218 or 219 or 221 or 244 or 246 )'

In [15]:
sample_dir = os.path.dirname(smina_top1_protein)
w = show_nv_traj(sample_dir, flex_resnumber, True)
w

NGLWidget(max_frame=20)

In [16]:
sample_dir = os.path.dirname(mdn_top1_protein)
w = show_nv_traj(sample_dir, flex_resnumber, True)
w

NGLWidget(max_frame=20)

### 🎉🎉End

Thanks for your interest in DiffBindFR. We are still working hard to further improve performance and extend it to other applications.

If you have any question, feel free to open a [github issue](https://github.com/HBioquant/DiffBindFR/issues) or reach out to me: [zhujt@stu.pku.edu.cn](zhujt@stu.pku.edu.cn)

👋👋👋